# 部署服务器
##  ubuntu系统
### 数据库
- sudo apt update 更新安装包
- apt install mysql-server mysql-client

- 设置远程访问mysql
    - 修改配置文件
    - /etc/mysql/mysql.conf.d
    - 查找 mysql.conf  find / -name mysql.cnf
    - 注释mysql.cof文件的bind_address
    - 进入数据库操作
    - mysql -u root -p
    - GRANT ALL PRIVILEGES ON *.* TO '数据库账号'@'%' IDENTIFIED BY '数据库密码' WITH GRANT OPTION;
    - 更新数据
    - flush privileges; 
    - 退出数据库，重启数据库
    - service mysql restart
    - 如果是阿里云要在安全规则里添加3306，TCP的端口
    

###  修改django的配置文件
- settings.py中
    - DEBUG=FALSE，ALLOWED_HOST=['*']
    - STATIC_ROOT = os.path.join(BASE_DIR, 'static')
    

In [ ]:
"""
 urls.py
         from django.views.static import serve
         url中加入以下配置
         url(r'^static/(?P<path>.*)$', serve, {"document_root": settings.STATIC_ROOT}),
         url(r'^media/(?P<path>.*)$', serve, {"document_root": settings.MEDIA_ROOT}),
         url(r'^$', views.home)

"""

### 服务器中
- 建文件夹，注意一定要在根目录下面创建不知道是否是在根目录可以用pwd查看
- 文件结构如下
- home
    - app
        - conf 将配置文件放在这个文件夹下
        - log 将日志放在这个文件夹下
        - src 将项目放在这个文件夹下
- 安装pip3
- apt install python3-pip

- 安装必备库
	- pip3 install django==1.11
	- pip3 install pymysql
	- pip3 install Pillow
   - pip3 install djangorestframework==3.4.6
   - pip3 install django-filter

- 查看进程
    - netstat -lntp

-  启动项目
    - python3 manage.py runserver 0.0.0.0:8000
    - 注意后面端口，需要在服务器上开对应的端口
- 通过这种方式就可以在服务器上部署了项目，但实际中有更好的部署方式


## 部署方法2：
### 使用nginx+uwsgi配置django项目

-  安装nginx
    - sudo apt-get install nginx
	
- 查看nginx的状态

    - systemctl status nginx 查看nginx的状态
    - systemctl start/stop/enable/disable nginx 启动/关闭/设置开机启动/禁止开机启动
    - service nginx status/stop/restart/start

-  安装uwsgi
    - pip3 install uwsgi

- nginx的配置文件中加载自定义的nginx的配置文件

    - 在/home/app/conf/下创建一个自定义的conf文件
    - 内容如下

In [ ]:
"""
server {
        listen 80; # 监听80端口
        server_name 47.105.32.183 localhost; #服务器地址

        access_log /home/app/log/access.log; # 日志路径
        error_log /home/app/log/error.log; # 错误日志路径

        location / {
            include uwsgi_params; # 将动态的求其交给uwsgi处理
            uwsgi_pass 127.0.0.1:8090; # uwsgi端口，注意与下面的一致
        }

        location /static/ { # 将静态文件交由nginx来处理
            alias /home/app/src/aixianfeng/static/; 
            expires 30d;
        }
}
"""

- 在 /etc/nginx/nginx.conf中修改配置
    - 找到有include的地方，加入下面的代码
    - include /home/app/conf/*.conf;

- 在/home/app/conf/下创建一个自定义的uwsgi.ini文件

- 内容如下

In [ ]:
"""
# 注意下面的这个文件里不要用#注释
[uwsgi]

master = true

processes = 4

pythonpath = /home/app/src/aixianfeng

module = aixianfeng.wsgi

socket = 127.0.0.1:8090

logto = /home/app/log/uwsgi.log
"""

- 启动方式：在当前文件夹下 uwsgi --ini uwsgi.ini